In [19]:
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import librosa
import awkward as ak
import matplotlib.pyplot as plt
from scipy.signal import decimate
from IPython.display import Audio 

In [20]:
dataset_folder = pathlib.Path("DATASETS/Audios") 

In [21]:
file_list = list(dataset_folder.iterdir())
for i, path in enumerate(file_list):
    if path.suffix != ".wav":
        file_list.pop(i)
file_list = sorted(file_list)
file_list

[WindowsPath('DATASETS/Audios/03-01-01-01-01-01-01.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-02.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-03.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-04.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-05.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-06.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-07.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-08.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-09.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-10.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-11.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-12.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-13.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-14.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-15.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-16.wav'),
 WindowsPath('DATASETS/Audios/03-01-01-01-01-01-17.wav'),
 WindowsPath('

In [71]:
ts, sr = librosa.load("DATASETS/Audios/03-02-02-02-02-02-09.wav", sr=None)
ts, _ = librosa.effects.trim(ts, top_db=20)
ts=decimate(ts, q=15)
print(ts.shape)
Audio(ts,rate=sr/15)

(15975,)


In [72]:
b = ak.ArrayBuilder()
for path in tqdm(file_list):
    librosa_audio_segment, sr = librosa.load(path, sr=None)
    librosa_audio_segment, _ = librosa.effects.trim(librosa_audio_segment, top_db=20)
    librosa_audio_segment=decimate(librosa_audio_segment, q=15)
    b.begin_list()
    b.begin_list()
    for value in librosa_audio_segment:
        b.real(value)
    b.end_list() 
    b.end_list() 
array = b.snapshot()

  0%|          | 0/2452 [00:00<?, ?it/s]

In [73]:
CATEGORICAL_FEATURES_NAMES = {
    "modality": {"01": "full-AV", "02": "video-only", "03": "audio-only"}, 
    "vocal_channel": {"01": "speech", "02": "song"},
    "emotion": {"01" : "neutral", "02" : "calm", "03" : "happy", "04" : "sad", "05" : "angry", "06" : "fearful", "07" : "disgust", "08" : "surprised"},
    "emotional_intensity": {"01" : "normal", "02" : "strong"},
    "statement": {"01" : "Kids are talking by the door", "02" : "Dogs are sitting by the door"},
    "repetition": {"01" : "1st", "02" : "2nd"},
    "actor": {str(i).zfill(2): str(i).zfill(2) for i in range(1, 25)}
}

In [74]:
df = list()
file_names = list()
for path in file_list:
    stem = path.stem.split("-")
    file_names.append(path.name)
    df.append(stem)
df = pd.DataFrame(df)
df.columns = list(CATEGORICAL_FEATURES_NAMES.keys())

for column in df.columns:
    df = df.replace(CATEGORICAL_FEATURES_NAMES)

df["sex"] = ["F" if i % 2 == 0 else "M" for i in df["actor"].astype(int)]
df["filename"] = file_names

In [75]:
df.head()

,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,filename
0,audio-only,speech,neutral,normal,Kids are talking by the door,1st,01,M,03-01-01-01-01-01-01.wav
1,audio-only,speech,neutral,normal,Kids are talking by the door,1st,02,F,03-01-01-01-01-01-02.wav
2,audio-only,speech,neutral,normal,Kids are talking by the door,1st,03,M,03-01-01-01-01-01-03.wav
3,audio-only,speech,neutral,normal,Kids are talking by the door,1st,04,F,03-01-01-01-01-01-04.wav
4,audio-only,speech,neutral,normal,Kids are talking by the door,1st,05,M,03-01-01-01-01-01-05.wav


In [76]:
df_sorted_actor = df.sort_values(by=["actor", "filename"])
idxs_sorted = list(df_sorted_actor.index)

In [77]:
idx_train = list(df_sorted_actor[df_sorted_actor.actor.astype(int) < 19].index)
idx_test = list(df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19].index)

In [78]:
X_train = array[idx_train]
X_test = array[idx_test]
Y_train = df_sorted_actor[df_sorted_actor.actor.astype(int) < 19]
Y_test = df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19]

In [79]:
def pad_X(X, m_max, nan_value=0):
    return ak.fill_none(ak.pad_none(X, m_max, axis=2, clip=True), value=nan_value)

In [80]:
# find the max length of X_train
maximum = 0
for ts in X_train:
    length = len(np.asarray(np.ravel(ts)))
    if length > maximum:
        maximum = length
maximum

15975

In [81]:
X_train = np.squeeze(np.array(pad_X(X_train, maximum, np.nan)))
X_test = np.squeeze(np.array(pad_X(X_test, maximum, np.nan)))

In [82]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1828, 15975)
(624, 15975)
(1828, 9)
(624, 9)


## Saving

In [83]:
np.save("DATASETS/TimeSeriesTrimmedDonwsapled/X_train15.npy", X_train)
np.save("DATASETS/TimeSeriesTrimmedDonwsapled/X_test15.npy", X_test)

In [84]:
Y_train.to_csv("DATASETS/TimeSeriesTrimmedDonwsapled/y_train.csv", index=False)
Y_test.to_csv("DATASETS/TimeSeriesTrimmedDonwsapled/y_test.csv", index=False)

### Load time series

In [23]:
X_train = np.load("DATASETS/TimeSeries/RavdessTimeSeries_X_train.npy")
X_test = np.load("DATASETS/TimeSeries/RavdessTimeSeries_X_test.npy")
print(X_train.shape, X_test.shape)

(1828, 304304) (624, 304304)


In [27]:
X_train

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  0.00000000e+00, -3.05175781e-05, ...,
                    nan,             nan,             nan],
       [ 3.05175781e-05,  3.05175781e-05,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
                    nan,             nan,             nan]])